In [1]:
## Loading Libraries
import pandas as pd
import numpy as np
import sys
import jdatetime as jd
import os
import pickle

# import power_index_calculator as px
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
## Loading functions
from convert_ar_characters import convert_ar_characters
from ReadingBalancesheetData import read_blnc_data
from nthMax import nth_max
from gini import gini
from find_shapley import find_shapley
from find_banzhaf import find_banzhaf
from gameTheoric_concentration import gameTheoric_concentration
from fill_shapley_banzhaf import fill_shapley_banzhaf

# **Loading Shareholder Data**

In [3]:
year = 1399
file_name_holderdata = "Shareholder"+str(year)+".csv"
file_name_measures = "Measures"+str(year)+".csv"
file_name_blnc = str(year)[2:]+'.txt'
file_name_measures_blnc = 'data_owenership'+str(year)[2:]+'_blnc'+str(year)[2:]+'.csv'

In [5]:
## Loading DATA
os.chdir(r"C:\Users\Mahdi\OneDrive\Master Thesis\Data")
SDATA = pd.read_csv(file_name_holderdata,index_col=0).drop_duplicates()

In [6]:
# Creating Dataframe for saving concentration mearsurs
CMdf = SDATA.groupby('Symbol',as_index=False).agg({'Id_tse':'first','Industry':'first','percent':'sum',
                                                   'ShareHolder':'count','MarketCap':'first'}).rename(columns=
                                                            {'ShareHolder':'Num_holders','percent':'sum_over1'})
CMdf.reset_index(drop=True,inplace=True)

## Filtering Data

In [7]:
## Filtering Data
print('Number of Observation is: ',len(CMdf),'\n')
theta_sum1 = 95
theta_holder = 1
MC = 15e10


filtered = CMdf[CMdf.sum_over1>=theta_sum1]
CMdf = CMdf[CMdf.sum_over1<=theta_sum1]
print(len(filtered),' is deleted becaause their sum_above1% is equal to or more than ',theta_sum1,'%')

filtered = CMdf[CMdf.Num_holders<=theta_holder]
CMdf = CMdf[CMdf.Num_holders>theta_holder]
print(len(filtered),' is deleted becaause their number of holder is less or equal than ',theta_holder)

filtered = CMdf[CMdf.MarketCap<=MC]
CMdf = CMdf[np.logical_or(CMdf.MarketCap>MC,pd.isnull(CMdf.MarketCap))]
print(len(filtered),' is deleted becaause their MarketCap is less or equal than ',MC)

print('\nNumber of remained Observation is: ',len(CMdf))

Number of Observation is:  727 

75  is deleted becaause their sum_above1% is equal to or more than  95 %
75  is deleted becaause their number of holder is less or equal than  1
0  is deleted becaause their MarketCap is less or equal than  150000000000.0

Number of remained Observation is:  577


# **Concentration Measures**

## 1- Largest Owner

In [8]:
temp = SDATA.groupby('Symbol',as_index=False).agg({'percent':'max'}).rename(columns={'percent':'Largest_Owner'})
CMdf = pd.merge(CMdf,temp,left_on='Symbol',right_on='Symbol',how='left')

## 2- First/Second

In [9]:
temp = SDATA.groupby('Symbol',as_index=False).agg({'percent':{lambda x: max(x)/nth_max(x,nth=2,interval=False)}}).rename(columns={'percent':'First_Second'})
CMdf = pd.merge(CMdf,temp,left_on='Symbol',right_on='Symbol',how='left').rename(columns={('First_Second', '<lambda>'):'First_Second'})

C:\Users\Mahdi\Anaconda3\lib\site-packages\pandas\core\reshape\merge.py:618: UserWarning: merging between different levels can give an unintended result (1 levels on the left, 2 on the right)
  warnings.warn(msg, UserWarning)
C:\Users\Mahdi\Anaconda3\lib\site-packages\pandas\core\generic.py:3936: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  obj = obj._drop_axis(labels, axis, level=level, errors=errors)


## 3- First/Sumtwofour

In [10]:
temp = SDATA.groupby('Symbol',as_index=False).agg({'percent':{lambda x: max(x)/sum(nth_max(x,nth=[2,4],interval=True))}}).rename(
    columns={'percent':'First_Sumtwofour'})
CMdf = pd.merge(CMdf,temp,left_on='Symbol',right_on='Symbol',how='left').rename(columns={('First_Sumtwofour', '<lambda>'):'First_Sumtwofour'})

## 4- Sumfive

In [11]:
temp = SDATA.groupby('Symbol',as_index=False).agg({'percent':{lambda x: sum(nth_max(x,nth=[1,5],interval=True))}}).rename(columns={'percent':'Sumfive'})
CMdf = pd.merge(CMdf,temp,left_on='Symbol',right_on='Symbol',how='left').rename(columns={('Sumfive', '<lambda>'):'Sumfive'})

## 5- Gini

We calculate gini coeficient using Deaton 1997:
$$ \gamma = \frac{N+1}{N-1} - \frac{2}{\mu\times N\times(N-1)}\sum_{i=1}^N{\rho_ix_i}$$

In [12]:
temp = SDATA.groupby('Symbol',as_index=False).agg({'percent':{lambda x: gini(list(x))}}).rename(columns={'percent':'Gini'})
CMdf = pd.merge(CMdf,temp,left_on='Symbol',right_on='Symbol',how='left').rename(columns={('Gini', '<lambda>'):'Gini'})

## 6- Herfindhal

In [13]:
temp = SDATA.groupby('Symbol',as_index=False).agg({'percent':{lambda x: sum([(t/100)**2 for t in list(x)])}}).rename(columns={'percent':'Herfindhal'})
CMdf = pd.merge(CMdf,temp,left_on='Symbol',right_on='Symbol',how='left').rename(columns={('Herfindhal', '<lambda>'):'Herfindhal'})

## 7 & 8-Shapley-Shubik & Banzhaf

In [14]:
# # Initiating columns
# CMdf['SSCL'] = np.nan
# CMdf['SSCO'] = np.nan
# CMdf['SSDL'] = np.nan
# CMdf['SSDO'] = np.nan
# CMdf['BZCL'] = np.nan
# CMdf['BZCO'] = np.nan
# CMdf['BZDL'] = np.nan

os.chdir(r"C:\Users\Mahdi\OneDrive\Master Thesis\Data")
CMdf_load = pd.read_csv(file_name_measures)

CMdf = pd.merge(CMdf,CMdf_load[['Symbol','SSCL', 'SSCO', 'SSDL', 'SSDO', 'BZCL', 'BZCO', 'BZDL']],left_on='Symbol',right_on='Symbol',how='left')

In [15]:
%%time
data = fill_shapley_banzhaf(data = CMdf,SDATA=SDATA,fast_mode = True,time_pnt=True,major_thr = 10)
CMdf = data['CMdf']

print('len(Errors): ',len(data['Errors']))
data['Errors']
[x for x in data['Errors'] if x[2]!= 'Error: request error!']

The symbol  آریان  mode:  SSDL ,  1  from  7
The symbol  تکالا  mode:  SSDL ,  2  from  7
The symbol  سپرده  mode:  SSDL ,  3  from  7
The symbol  نبورس  mode:  SSDL ,  4  from  7
The symbol  نکالا  mode:  SSDL ,  5  from  7
The symbol  وپارس  mode:  SSDL ,  6  from  7
The symbol  کیان  mode:  SSDL ,  7  from  7


**About  0  out of  7  of mising cells are filled!
len(Errors):  7
Wall time: 7min 42s


[]

In [17]:
os.chdir(r"C:\Users\Mahdi\OneDrive\Master Thesis\Data")
CMdf.to_csv(file_name_measures)

# **Loading Balancesheet Data and Merging**

In [13]:
blnc_data = read_blnc_data(file=file_name_blnc)
blnc_data['book_value'] = blnc_data.Tot_asset-blnc_data.Tot_lib
blnc_data = blnc_data[blnc_data.book_value>0]

In [14]:
data_out = pd.merge(CMdf,blnc_data[['Symbol','Tot_asset', 'Tot_lib', 'Capital', 'Equity_at_year_end','Debt_Equity', 'Debt_Equity_normal', 'book_value']]
                    ,left_on='Symbol',right_on='Symbol',how='outer')

os.chdir(r"C:\Users\Mahdi\OneDrive\Master Thesis\Data")
# data_out.to_csv('data_owenership98_blnc98.csv')